# Container Apps

This notebook contains 2 demos showcasing Container Apps

--------------------

## Demo 1 - Simple setup with **microservices** and **Dapr** communication

Prerequisites

----

* [Azure Subscription](https://azure.microsoft.com/en-au/free/)
* [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli)
* [k6](https://k6.io/docs/getting-started/installation/) for generating orders

---
Architecture


![](./node-python/img/Architecture_Diagram.png)


  

First we setup base resources:
```bash

name=aca$(cat /dev/urandom | tr -dc '[:lower:]' | fold -w ${1:-5} | head -n 1)
resourceGroup="rg-nodepython-${name}"
location="northeurope"
acaEnv="env-${name}"
```

```bash
az login

az extension add --name containerapp
az provider register --namespace Microsoft.App
az provider show -n Microsoft.App --query registrationState

az group create \
  --name $resourceGroup \
  --location "$location"

az deployment group create \
  --name env-create \
  -g $resourceGroup \
  --template-file ./environment.bicep \
  --parameters environmentName=$acaEnv

az containerapp env list -o table
az containerapp env show -n $acaEnv -g $resourceGroup
az containerapp env dapr-component list -n $acaEnv -g $resourceGroup -o table
az containerapp env dapr-component show -n $acaEnv -g $resourceGroup --dapr-component-name statestore

deployOutputs=$(az deployment group show --name env-create -g $resourceGroup --query properties.outputs)

acrUserName=$(echo $deployOutputs | jq -r .acrUserName.value)
acrPassword=$(echo $deployOutputs | jq -r .acrPassword.value)
acrloginServer=$(echo $deployOutputs | jq -r .acrloginServer.value)
acrName=$(echo $ACR_LOGIN_SERVER | cut -f1,1 -d .)
workspaceId=$(echo $deployOutputs | jq -r .workspaceId.value)
```


## Create new container images 
When published, go to the nodeapp ingress URL/ports to see version

---
```bash

cd containers/node

az acr build --image hello-aca-node:v1 \
  --registry $acrName \
  --file Dockerfile .

cd ../python

az acr build --image hello-aca-python:v1 \
  --registry $acrName \
  --file Dockerfile .
  ```